In [9]:
!pip install langchain openai

### LLMChain

In [8]:
from langchain import PromptTemplate, OpenAI, LLMChain
import os 

os.environ["OPENAI_API_KEY"] = "Enter your api key"

prompt_template = "What is capital of {country}?"

llm = OpenAI(temperature = 0)
llm_chain = LLMChain(
  llm = llm,
  prompt = PromptTemplate.from_template(prompt_template)
)

llm_chain("Egypt")

{'country': 'Egypt', 'text': '\n\nCairo is the capital of Egypt.'}

### Sequential Chain
### Ardışık Zincir 

#### Simple Sequential Chain 
#### Basit Ardışık Zincir

In [10]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.9)


# Prompt 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the capital of {country}?"
)
chain_one = LLMChain(llm= llm, prompt = first_prompt)

# Prompt 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    country: {capital}"
)
chain_two = LLMChain(llm= llm, prompt=second_prompt)

# Simple sequential chain / Basit Ardışık Zincir
simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True   # This line shows the process / Bu satır süreci gösterir
)

simple_chain.run("Egypt")



> Entering new SimpleSequentialChain chain...
The capital of Egypt is Cairo.
Egypt is a historic country with Cairo as its capital, known for its iconic pyramids and ancient civilization.

> Finished chain.


'Egypt is a historic country with Cairo as its capital, known for its iconic pyramids and ancient civilization.'

In [11]:
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.9)

# Prompt + Chain 1 
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to Arabic:"
    "\n\n{Review}"
)
chain_one = LLMChain(llm= llm, prompt= first_prompt, output_key="Arabic_review")

# Prompt + Chain 2 
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{Arabic_review}"
)
chain_two = LLMChain(llm= llm, prompt= second_prompt, output_key= "summary")

# Prompt + Chain 3
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain_three = LLMChain(llm= llm, prompt= third_prompt, output_key="language")

# Prompt + Chain 4
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm= llm, prompt= fourth_prompt, output_key="followup_message")

# Sequential Chain
# Ardışık Zincir
seq_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["Arabic_review", "summary","followup_message"],
    verbose=True
)

review = "Yesterday I bought a T-shirt from addidas it was colorful with pretty good matrial but the size was very large"
seq_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Yesterday I bought a T-shirt from addidas it was colorful with pretty good matrial but the size was very large',
 'Arabic_review': 'أمس اشتريت قميصًا من أديداس، كان ملونًا ومصنوعًا من مادة جيدة جدًا، لكن الحجم كان كبيرًا جدًا.',
 'summary': 'Yesterday I bought a colored Adidas shirt made of very good material, but the size was way too big.',
 'followup_message': 'Thank you for your feedback about your recent purchase. We appreciate hearing your thoughts. We apologize that the size of the colored Adidas shirt you bought was way too big for you. We strive to provide accurate sizing information, and we apologize if this was not the case in your particular order. We will take note of this issue and work on improving our sizing options in the future. If you would like to return or exchange the product, kindly reach out to our customer service team, and they will be happy to assist you. Thank you again for bringing this to our attention, and we hope to serve you better in your fu

### Router Chain
### Yönlendirici Zincir

In [12]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [13]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [15]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [16]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [17]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has various applications, including understanding the behavior of stars, explaining the cosmic microwave background radiation, and developing technologies like incandescent light bulbs and thermal imaging devices.'

In [18]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for your kind words! As a mathematician, I can definitely help you with your question.\n\nThe problem you presented is a simple addition problem: 2 + 2. To solve it, we can add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "what is 2 + 2?" is 4.'

In [19]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


/opt/conda/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next during reproduction. Therefore, every cell in our body contains DNA to ensure the proper functioning and continuity of life.'